# Visualize FFT Datasets

Visualize the real TNG FFT datasets and the TVB sim FFT datasets.

In [1]:
import sys
# sys.path.append('../../dnn/')
sys.path.append('../dnn/')
import time
import numpy as np
# np.random.seed(1234)
import math as m
import os

import processing
import processing.preprocessfft
from processing.util import DataHandler
import peakdetect

# import DNN frameworks
import tensorflow as tf
import keras
from sklearn.decomposition import PCA

import ntpath

%load_ext autoreload
%autoreload 2

/Users/adam2392/Documents/dnn-unsupervised/.venv/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
# load in the data



In [3]:
# load in the image type data


In [4]:
# plot and visualize the fft data 




In [5]:
# plot and visualize the image data compared real vs sim


